In [1]:
from problema.problema import Problema
import pulp as pu
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px


In [2]:
problema = Problema(file='model_template_1205v2.xlsm')

Construyendo variables de planta


100%|██████████| 13/13 [00:00<00:00, 390.78it/s]


Construyendo componentes del modelo de planta


100%|██████████| 13/13 [00:00<00:00, 16.74it/s]


Construyendo componentes de material llegando


100%|██████████| 8/8 [00:00<00:00, 3057.07it/s]


Construyendo componentes de inventario en puerto


100%|██████████| 62/62 [00:00<00:00, 406.24it/s]


Agregando restricciones al modelo


100%|██████████| 10451/10451 [00:00<00:00, 159406.34it/s]


In [3]:
problema.solve(t_limit_minutes=20)

Cantidad de cpu: 15
gap: 1000000
t_limit: 20 minutos
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/luispinilla/Documents/source_code/bios_solver/env/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0p/md4rzs_n7bg992nr5mzyl7_00000gn/T/3cc960ed89f24bce8278ee0f6aec8b67-pulp.mps sec 1200 allow 1000000 threads 15 timeMode elapsed branch printingOptions all solution /var/folders/0p/md4rzs_n7bg992nr5mzyl7_00000gn/T/3cc960ed89f24bce8278ee0f6aec8b67-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10456 COLUMNS
At line 174354 RHS
At line 184806 BOUNDS
At line 207529 ENDATA
Problem MODEL has 10451 rows, 35928 columns and 87520 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 1200
allowableGap was changed from 1e-10 to 1e+06
threads was changed from 0 to 15
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2.47574e+10 - 0.10 seconds


'Optimal'

In [4]:
transporte_df = problema.reporte_transporte()
reporte_plantas_df = problema.reporte_planta()


In [6]:
puerto_df = reporte_puerto(problema)

In [13]:
def dibujar_inventario_ingrediente_puerto(df: pd.DataFrame, puerto:str, ingrediente: str) -> go.Figure:
    fig = go.Figure()

    temp = df[(df['ingrediente'] == ingrediente) & (df['puerto'] == puerto)].copy()

    temp['key'] = temp['empresa'] + ", " + temp['importacion']

    temp = temp.pivot_table(index='fecha', columns='key', values='inventario_kg', aggfunc='sum').reset_index()

    for importacion in temp.drop(columns=['fecha']).columns:
        fig.add_trace(go.Scatter(
            x=temp['fecha'],  # x-axis
            y=temp[importacion],  # y-axis
            mode='lines',  # Connect data points with lines
            name=importacion  # Name in the legend
        )
        )

    # Layout parameters
    fig.update_layout(
        title=f'Puerto: {puerto}, Ingrediente: {ingrediente}',  # Title
        xaxis_title='fecha',  # y-axis name
        yaxis_title='Kg',  # x-axis name
        xaxis_tickangle=0,  # Set the x-axis label angle
        showlegend=True,     # Display the legend
        barmode='stack',
    )
    return fig


In [8]:
for puerto in puerto_df['puerto'].unique():
    fig = dibujar_inventario_ingrediente_puerto(df=puerto_df, puerto=puerto, ingrediente='maiz')
    fig.show()

In [9]:
def dibujar_inventario_puerto(df: pd.DataFrame, empresa:str, puerto:str, ingrediente:str, importacion:str) -> go.Figure:
    
    df['despachos_indirectos_kg'] = 34000*df['despachos_indirectos']

    temp = df[(df['empresa'] == empresa) &
              (df['ingrediente'] == ingrediente) &
              (df['importacion']==importacion)].copy()


    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=temp['fecha'],  # x-axis
        y=temp['inventario_kg'],  # y-axis
        mode='lines',  # Connect data points with lines
        name='Inventario'  # Name in the legend
    )
    )

    fig.add_trace(go.Bar(
        x=temp['fecha'], 
        y=temp['ingreso_bodega'],
        name='Ingreso a Bodega')
    )

    fig.add_trace(go.Bar(
        x=temp['fecha'], 
        y=temp['despachos_indirectos_kg'],
        name='Despacho hacia planta',
        hovertext=temp['plantas'])
    )

    # Layout parameters
    fig.update_layout(
        title=f'Puerto: {puerto}, Empresa: {empresa}, Ingrediente: {ingrediente}, Importacion: {importacion}',  # Title
        xaxis_title='Fecha',  # y-axis name
        yaxis_title='Kg',  # x-axis name
        xaxis_tickangle=0,  # Set the x-axis label angle
        showlegend=True,     # Display the legend
        barmode='stack',
        legend=dict(orientation='h',  x=0.5, xanchor='center'),
    )

    return fig

In [10]:
grupo_puerto = puerto_df.groupby(by=['puerto', 'empresa', 'importacion', 'ingrediente'])[
    ['inventario_kg']].sum().reset_index()
grupo_puerto = grupo_puerto[grupo_puerto['inventario_kg'] > 0]
grupo_puerto

for i in grupo_puerto.index:
    puerto = grupo_puerto.loc[i]['puerto']
    empresa = grupo_puerto.loc[i]['empresa']
    importacion = grupo_puerto.loc[i]['importacion']
    ingrediente = grupo_puerto.loc[i]['ingrediente']

    fig = dibujar_inventario_puerto(df=puerto_df,
                                    empresa=empresa,
                                    puerto=puerto,
                                    ingrediente=ingrediente,
                                    importacion=importacion)
    fig.show()

In [11]:
def dibujar_planta(df: pd.DataFrame, nombre_planta: str, ingrediente: str):

    temp = df[(df['planta'] == nombre_planta) &
              (df['ingrediente'] == ingrediente)]

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=temp['fecha'],  # x-axis
        y=temp['inventario'],  # y-axis
        mode='lines',  # Connect data points with lines
        name='Inventario'  # Name in the legend
    )
    )

    fig.add_trace(go.Scatter(
        x=temp['fecha'],  # x-axis
        y=temp['capacidad'],  # y-axis
        mode='lines',  # Connect data points with lines
        name='Capacidad Almacenamiento',  # Name in the legend
        line=dict(dash='dash', color='blue')
    ))

    fig.add_trace(go.Bar(
        x=temp['fecha'],
        y=temp['llegada_bodega'],
        name='Despacho Bodega',
        marker_color='green',
        hovertext=temp['importaciones'])
    )

    fig.add_trace(go.Bar(
        x=temp['fecha'],
        y=temp['llegada_directa'],
        name='Despacho Directo',
        marker_color='orange',
        hovertext=temp['importaciones'])
    )

    fig.add_trace(go.Scatter(
        x=temp['fecha'],  # x-axis
        y=temp['safety_stock'],  # y-axis
        mode='lines',  # Connect data points with lines
        name='Safety Stock',  # Name in the legend
        line=dict(dash='dash', color='red')
    ))

    fig.add_trace(go.Bar(
        x=temp['fecha'],  # x-axis
        y=temp['llegada_programada'],  # y-axis
        name='Llegadas Programada',  # Name in the legend
        marker_color='gray'
    ))

    fig.add_trace(go.Bar(
        x=temp['fecha'],  # x-axis
        y=temp['backorder'],  # y-axis
        name='Backorder',  # Name in the legend
        marker_color='red'
    ))

    # Layout parameters
    fig.update_layout(
        title=f'Planta: {nombre_planta}: {ingrediente}',  # Title
        xaxis_title='Fecha',  # y-axis name
        yaxis_title='Kg',  # x-axis name
        xaxis_tickangle=0,  # Set the x-axis label angle
        showlegend=True,     # Display the legend
        barmode='stack',
        legend=dict(orientation='h',  x=0.5, xanchor='center'),
    )
    return fig

In [12]:
for nombre, planta in problema.plantas.items(): 
    fig = dibujar_planta(df=reporte_plantas_df, nombre_planta=nombre, ingrediente='maiz')
    fig.show()